- `Hardware Accelerator > GPU`

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/gpu-step2.jpg" alt="GPU Step 2">

In [ ]:
!apt install python-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay

In [1]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [ ]:
!python3 -m pip install -U pip

In [ ]:
!pip install -U numpy==1.20

In [ ]:
!pip install -U stable-baselines3[extra]
!pip install -U gymnasium

In [ ]:
!pip install -U huggingface_sb3
!pip install -U huggingface_hub
!pip install -U panda_gym

In [2]:
import os

import gymnasium as gym
import panda_gym

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

from huggingface_hub import notebook_login

/usr/local/lib/python3.8/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [3]:
env_id = "PandaReachDense-v3"

# Create the env
env = gym.make(env_id)

# Get the state space and action space
s_size = env.observation_space.shape
a_size = env.action_space

pybullet build time: Nov 28 2023 23:51:11


argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


In [4]:
print("_____OBSERVATION SPACE_____ \n")
print("The State Space is: ", s_size)
print("Sample observation", env.observation_space.sample()) # Get a random observation

_____OBSERVATION SPACE_____ 

The State Space is:  None
Sample observation OrderedDict([('achieved_goal', array([0.31196994, 9.922005  , 0.9234746 ], dtype=float32)), ('desired_goal', array([9.436451 , 9.806625 , 7.8390083], dtype=float32)), ('observation', array([ 6.817663 ,  1.5526605, -8.763252 ,  3.483585 ,  2.8651123,
       -6.2117634], dtype=float32))])


In [5]:
print("\n _____ACTION SPACE_____ \n")
print("The Action Space is: ", a_size)
print("Action Space Sample", env.action_space.sample()) # Take a random action


 _____ACTION SPACE_____ 

The Action Space is:  Box(-1.0, 1.0, (3,), float32)
Action Space Sample [-0.9920801   0.2645061   0.76553977]


In [6]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886


In [7]:
model = A2C(
    policy="MultiInputPolicy",
    env=env,
    verbose=1
)

Using cuda device


In [9]:
model.learn(1_000_000)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 42.3     |
|    ep_rew_mean        | -10.6    |
|    success_rate       | 0.174    |
| time/                 |          |
|    fps                | 463      |
|    iterations         | 100      |
|    time_elapsed       | 4        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -4.26    |
|    explained_variance | 0.874    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -1.47    |
|    std                | 1        |
|    value_loss         | 0.313    |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 42.6     |
|    ep_rew_mean        | -10.2    |
|    success_rate       | 0.196    |
| time/                 |          |
|    fps                | 489      |
|    iterations         | 200      |
|

In [10]:
# Save the model and  VecNormalize statistics when saving the agent
model.save("a2c-PandaReachDense-v3")
env.save("vec_normalize.pkl")

In [11]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

eval_env = DummyVecEnv([lambda: gym.make("PandaReachDense-v3")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

eval_env.render_mode = "rgb_array"

eval_env.training = False
eval_env.norm_reward = False

model = A2C.load("a2c-PandaReachDense-v3", eval_env)

mean_reward, std_reward = evaluate_policy(model, eval_env)
print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

argv[0]=--background_color_red=0.8745098114013672
argv[1]=--background_color_green=0.21176470816135406
argv[2]=--background_color_blue=0.1764705926179886
Mean reward = -0.24 +/- 0.12


/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [12]:
import os
from pathlib import Path
import tempfile

import numpy as np
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecVideoRecorder
)


def generate_replay(
    model,
    eval_env,
    video_length: int = 1000,
    is_deterministic: bool = True,
    local_path: Path = './',
    file_name = 'replay.mp4'
):
    """
    Generate a replay video of the agent
    :param model: trained model
    :param eval_env: environment used to evaluate the agent
    :param video_length: length of the video (in timesteps)
    :param is_deterministic: use deterministic or stochastic actions
    :param local_path: path of the local repository
    """
    # This is another temporary directory for video outputs
    # SB3 created a -step-0-to-... meta files as well as other
    # artifacts which we don't want in the repo.
    with tempfile.TemporaryDirectory() as tmpdirname:
        # Step 1: Create the VecVideoRecorder
        env = VecVideoRecorder(
            eval_env,
            tmpdirname,
            record_video_trigger=lambda x: x == 0,
            video_length=video_length,
            name_prefix="",
        )

        obs = env.reset()
        lstm_states = None
        episode_starts = np.ones((env.num_envs,), dtype=bool)

        try:
            for _ in range(video_length):
                action, lstm_states = model.predict(
                    obs,
                    state=lstm_states,
                    episode_start=episode_starts,
                    deterministic=is_deterministic,
                )
                obs, _, episode_starts, _ = env.step(action)

            # Save the video
            env.close()

            # Convert the video with x264 codec
            inp = env.video_recorder.path
            out = os.path.join(local_path, file_name)
            os.system(f"ffmpeg -y -i {inp} -vcodec h264 {out}".format(inp, out))

        except KeyboardInterrupt:
            pass
        except Exception as e:
            print(str(e))

In [14]:
generate_replay(
    model,
    eval_env,
    file_name='5-actor-critic.mp4'
)

Saving video to /tmp/tmph0cg5vxt/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmph0cg5vxt/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmph0cg5vxt/-step-0-to-step-1000.mp4



ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

Moviepy - Done !
Moviepy - video ready /tmp/tmph0cg5vxt/-step-0-to-step-1000.mp4


frame= 1001 fps=720 q=-1.0 Lsize=     642kB time=00:00:39.92 bitrate= 131.7kbits/s speed=28.7x    
video:630kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 1.818326%
[libx264 @ 0x55966877fbc0] frame I:5     Avg QP:17.02  size:  7149
[libx264 @ 0x55966877fbc0] frame P:352   Avg QP:21.14  size:   409
[libx264 @ 0x55966877fbc0] frame B:644   Avg QP:29.79  size:   722
[libx264 @ 0x55966877fbc0] consecutive B-frames:  3.2% 19.6% 40.5% 36.8%
[libx264 @ 0x55966877fbc0] mb I  I16..4: 18.6% 64.8% 16.6%
[libx264 @ 0x55966877fbc0] mb P  I16..4:  0.1%  0.3%  0.2%  P16..4:  1.4%  0.5%  0.3%  0.0%  0.0%    skip:97.2%
[libx264 @ 0x55966877fbc0] mb B  I16..4:  0.1%  0.1%  0.2%  B16..8:  3.1%  1.9%  0.7%  direct: 0.2%  skip:93.6%  L0:54.0% L1:44.8% BI: 1.2%
[libx264 @ 0x55966877fbc0] 8x8 transform intra:50.7% inter:7.5%
[libx264 @ 0x55966877fbc0] coded y,uvDC,uvAC intra: 19.9% 5.7% 4.5% inter: 0.8% 0.0% 0.0%
[libx264 @ 0x55966877fbc0] i16 v,h,dc,p: 68% 11% 21%  0%
[libx2

In [15]:
%%html
<video controls autoplay><source src="./5-actor-critic.mp4" type="video/mp4"></video>